# Setup
This template can be used to analyze the construct validity of a given model's personality test data.

1. Specify your model's full results pickle file, JSON `admin_session`, and identifier (model pointer), below.
2. If you'd like to save the test scores for further analysis, specify a `SAVE_SCORES_FILENAME`.
3. Run this notebook in `personality_in_llms/analysis` 

In [ ]:
# path to directory containing psyborgs
# this default path should work if you've cloned the repo
PATH = "../"

# path of pickled results to be analyzed
PKL_PATH = "../results/" + "your_results_here.pkl"

# admin_session filename
ADMIN_SESSION_PATH = "../admin_sessions/" + "prod_run_01_external_rating.json"

# identifier for the model to be analyzed. must match the `model_id` field in
# the results file (e.g., "meta-llama/Llama-2-13b-chat-hf")
MODEL_POINTER = "mistralai/Mixtral-8x7B-Instruct-v0.1"

# save joined test scores? If no, leave `False`
SAVE_SCORES_FILENAME = False

## Load Dependencies

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import numpy as np
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
from rpy2.robjects.conversion import localconverter

import sys
sys.path.append(PATH)

from psyborgs import score_calculation, survey_bench_lib

## Unpickle Raw Results

In [ ]:
df_raw_response_scores = pd.read_pickle(PKL_PATH)


In [ ]:
df_raw_response_scores

In [ ]:
SPID = ['item_preamble_id',
        'item_postamble_id',
        'response_scale_id',
        'response_choice_postamble_id',
        'model_id']

BFI_SCALE_IDS = ["BFI-EXT", "BFI-AGR", "BFI-CON", "BFI-NEU", "BFI-OPE"]
IPIP_SCALE_IDS = ["IPIP300-EXT", "IPIP300-AGR", "IPIP300-CON", "IPIP300-NEU", "IPIP300-OPE"]
VALIDATION_SCALE_IDS = ["PA", "NA", "CSE", "CPI", "PHYS", "VRBL", "ANGR", "HSTL", "ACHV", "CONF", "SCRT"]

In [ ]:
test_df = df_raw_response_scores.query(
    "item_postamble_id == 'plk-bfi-0' & item_preamble_id == 'd33-th2' & item_id == 'bf1'"
)

test_df

## Load Admin Session


In [ ]:
admin_session = survey_bench_lib.load_admin_session(
    ADMIN_SESSION_PATH)

# Score Session

In [ ]:
# adapt df to match a df with scores for possible continuations
df_raw_response_scores['score'] = 1
df_raw_response_scores['response_value'] = df_raw_response_scores['model_output'].astype('int')

In [ ]:
# score session
scored_session_df = score_calculation.score_session(
    admin_session, df_raw_response_scores)

scored_session_df.head(5)

# Descriptives

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

### BFI and PANAS

In [ ]:
scored_session_df[["BFI-EXT", "BFI-AGR", "BFI-CON", "BFI-NEU", "BFI-OPE", "PA", "NA"]].describe().round(2)

In [ ]:
fig = plt.figure(tight_layout=True)
scored_session_df \
    [["BFI-EXT", "BFI-AGR", "BFI-CON", "BFI-NEU", "BFI-OPE"]] \
    .hist(range=[1,5], alpha=1, figsize=(10, 7.5), sharey=True)

plt.show()

### IPIP-NEO-300

In [ ]:
scored_session_df[["IPIP300-EXT", "IPIP300-AGR", "IPIP300-CON", "IPIP300-NEU", "IPIP300-OPE"]].describe().round(2)

In [ ]:
fig = plt.figure(tight_layout=True)
scored_session_df \
    [["IPIP300-EXT", "IPIP300-AGR", "IPIP300-CON", "IPIP300-NEU", "IPIP300-OPE"]] \
    .hist(range=[1,5], alpha=1, figsize=(10, 7.5), sharey=True)

plt.show()

### Criterion Measures

In [ ]:
scored_session_df[VALIDATION_SCALE_IDS].describe().round(2)

## Descriptives by Item Postamble

### IPIP-NEO-300

In [ ]:
original_postamble = scored_session_df[
    scored_session_df["item_postamble_id"].str.endswith("-0")] \
    [["IPIP300-EXT", "IPIP300-AGR", "IPIP300-CON", "IPIP300-NEU", "IPIP300-OPE"]]

postamble_3 = scored_session_df[
    scored_session_df["item_postamble_id"].str.endswith("-3")] \
    [["IPIP300-EXT", "IPIP300-AGR", "IPIP300-CON", "IPIP300-NEU", "IPIP300-OPE"]]

postamble_9 = scored_session_df[
    scored_session_df["item_postamble_id"].str.endswith("-9")] \
    [["IPIP300-EXT", "IPIP300-AGR", "IPIP300-CON", "IPIP300-NEU", "IPIP300-OPE"]]

postamble_103 = scored_session_df[
    scored_session_df["item_postamble_id"].str.endswith("-103")] \
    [["IPIP300-EXT", "IPIP300-AGR", "IPIP300-CON", "IPIP300-NEU", "IPIP300-OPE"]]

postamble_109 = scored_session_df[
    scored_session_df["item_postamble_id"].str.endswith("-109")] \
    [["IPIP300-EXT", "IPIP300-AGR", "IPIP300-CON", "IPIP300-NEU", "IPIP300-OPE"]]

In [ ]:
compare_postambles_df = pd.concat([original_postamble,
                                   postamble_3, postamble_9,
                                   postamble_103, postamble_109],
                                  keys=[0, 3, 9, 103, 109])

compare_postambles_df = compare_postambles_df.groupby(level=0)

# summary = compare_postambles_df.describe()
summary = compare_postambles_df.agg(['mean', 'std'])

summary

### Validation Measures

In [ ]:
original_postamble = scored_session_df[
    scored_session_df["item_postamble_id"].str.endswith("-0")] \
    [VALIDATION_SCALE_IDS]

postamble_3 = scored_session_df[
    scored_session_df["item_postamble_id"].str.endswith("-3")] \
    [VALIDATION_SCALE_IDS]

postamble_9 = scored_session_df[
    scored_session_df["item_postamble_id"].str.endswith("-9")] \
    [VALIDATION_SCALE_IDS]

postamble_103 = scored_session_df[
    scored_session_df["item_postamble_id"].str.endswith("-103")] \
    [VALIDATION_SCALE_IDS]

postamble_109 = scored_session_df[
    scored_session_df["item_postamble_id"].str.endswith("-109")] \
    [VALIDATION_SCALE_IDS]

compare_postambles_df = pd.concat([original_postamble,
                                   postamble_3, postamble_9,
                                   postamble_103, postamble_109],
                                  keys=[0, 3, 9, 103, 109])

compare_postambles_df = compare_postambles_df.groupby(level=0)

# summary = compare_postambles_df.describe()
summary = compare_postambles_df.agg(['mean', 'std'])

summary

PVQ-RR Achievement, Conformity, Security values are slightly lower when using the original item postamble.

# Criterion Validity

## Calculate p-values

In [ ]:

def calculate_pvalues(df):
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            tmp = df[df[r].notnull() & df[c].notnull()]
            pvalues[r][c] = round(pearsonr(tmp[r], tmp[c])[1], 4)
    return pvalues

## BFI Intercorrelations

In [ ]:
scored_session_df[BFI_SCALE_IDS].corr()

In [ ]:
calculate_pvalues(scored_session_df[BFI_SCALE_IDS])

## IPIP-NEO-300 Intercorrelations

In [ ]:
scored_session_df[IPIP_SCALE_IDS].corr()

In [ ]:
calculate_pvalues(scored_session_df[IPIP_SCALE_IDS])

## BPAQ Intercorrelations

In [ ]:
scored_session_df[["PHYS", "VRBL", "ANGR", "HSTL"]].corr()

In [ ]:
calculate_pvalues(scored_session_df[["PHYS", "VRBL", "ANGR", "HSTL"]])

## IPIP-NEO-300 Intercorrelations Across Item Postambles

In [ ]:
scored_session_df \
 .query("item_postamble_id == 'plk-ipip-0'") \
 [IPIP_SCALE_IDS].corr()

In [ ]:
scored_session_df \
 .query("item_postamble_id == 'plk-ipip-103'") \
 [IPIP_SCALE_IDS].corr()

In [ ]:
scored_session_df \
 .query("item_postamble_id == 'plk-ipip-109'") \
 [IPIP_SCALE_IDS].corr()

## PANAS Intercorrelations Across Item Postambles

In [ ]:
scored_session_df \
 .query("item_postamble_id == 'plk-panas-0'") \
 [["PA", "NA"]].corr()

In [ ]:
scored_session_df \
 .query("item_postamble_id == 'plk-panas-103'") \
 [["PA", "NA"]].corr()

In [ ]:
scored_session_df \
 .query("item_postamble_id == 'plk-panas-109'") \
 [["PA", "NA"]].corr()

# Join Data

In [ ]:
# simulated participant ID combo
SPID_2 = ['item_preamble_id',
          'item_postamble_id',
          'response_choice_postamble_id',
          'model_id']

scored_bfi = scored_session_df[
    SPID_2 + ["response_scale_id"] + BFI_SCALE_IDS
    ].query("item_postamble_id.str.contains(\'bfi')")

In [ ]:
# simulated participant ID combo
SPID_2 = ['item_preamble_id',
          'item_postamble_id',
          'response_choice_postamble_id',
          'model_id']

scored_bfi = scored_session_df[
    SPID_2 + ["response_scale_id"] + BFI_SCALE_IDS
    ].query("item_postamble_id.str.contains(\'bfi')")

scored_ipip = scored_session_df[
    SPID_2 + ["response_scale_id"] + IPIP_SCALE_IDS
    ].query("item_postamble_id.str.contains(\'ipip')")

scored_panas = scored_session_df[
    SPID_2 + [
        "response_scale_id", "PA", "NA"
    ]].query("item_postamble_id.str.contains(\'panas')")

scored_sscs = scored_session_df[
    SPID_2 + [
        "response_scale_id", "CSE", "CPI"
    ]].query("item_postamble_id.str.contains(\'sscs')")

scored_bpaq = scored_session_df[
    SPID_2 + [
        "response_scale_id", "PHYS", "VRBL", "ANGR", "HSTL"
    ]].query("item_postamble_id.str.contains(\'bpaq')")

scored_pvq = scored_session_df[
    SPID_2 + [
        "response_scale_id", "ACHV", "CONF", "SCRT"
    ]].query("item_postamble_id.str.contains(\'pvq')")

# create common postamble IDs
scored_bfi["common_item_postamble_id"] = scored_bfi.item_postamble_id.str.findall(r'\d+$').str[0]
scored_ipip["common_item_postamble_id"] = scored_ipip.item_postamble_id.str.findall(r'\d+$').str[0]
scored_panas["common_item_postamble_id"] = scored_panas.item_postamble_id.str.findall(r'\d+$').str[0]
scored_sscs["common_item_postamble_id"] = scored_sscs.item_postamble_id.str.findall(r'\d+$').str[0]
scored_bpaq["common_item_postamble_id"] = scored_bpaq.item_postamble_id.str.findall(r'\d+$').str[0]
scored_pvq["common_item_postamble_id"] = scored_pvq.item_postamble_id.str.findall(r'\d+$').str[0]

# join by common IDs
# simulated participant ID combo
SPID_3 = ['item_preamble_id',
          'common_item_postamble_id',
          'response_choice_postamble_id',
          'model_id']

# all scored DFs
all_scored_dfs = [scored_bfi, scored_ipip, scored_panas, scored_sscs, scored_bpaq, scored_pvq]

scored_joined = pd.merge(scored_bfi, scored_ipip, on=SPID_3, suffixes=('', '_DROP')) \
    .filter(regex='^(?!.*_DROP)') \
    .merge(scored_panas, on=SPID_3, suffixes=('', '_DROP')) \
    .filter(regex='^(?!.*_DROP)') \
    .merge(scored_sscs, on=SPID_3, suffixes=('', '_DROP')) \
    .filter(regex='^(?!.*_DROP)') \
    .merge(scored_bpaq, on=SPID_3, suffixes=('', '_DROP')) \
    .filter(regex='^(?!.*_DROP)') \
    .merge(scored_pvq, on=SPID_3, suffixes=('', '_DROP')) \
    .filter(regex='^(?!.*_DROP)')
# scored_joined = reduce(lambda left,right: pd.merge(left, right, left_index=True, right_index=True, how='outer', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)'), all_scored_dfs)

In [ ]:
scored_joined

In [ ]:
# optional: save scores to disk
if SAVE_SCORES_FILENAME:
    scored_joined.to_pickle(SAVE_SCORES_FILENAME)

# Joined Correlations

## BFI and IPIP-NEO-300

In [ ]:
scored_joined[BFI_SCALE_IDS + IPIP_SCALE_IDS].corr().round(2)

In [ ]:
calculate_pvalues(scored_joined[BFI_SCALE_IDS + IPIP_SCALE_IDS])

## Convergent & Discriminant Validity

In [ ]:
def get_avg_abs_discriminant_values(df):
    # use only absolute values
    abs_corr = abs(df)
    
    # get array length
    abs_corr_len = len(abs_corr)

    # get indices of diagonal elements
    dia = np.diag_indices(abs_corr_len)

    # get sum of diagonal elements
    dia_sum = sum(np.array(abs_corr)[dia])
    # dia_sum = sum(abs_corr[dia])    

    # get sum of off-diagonal elements
    off_dia_sum = np.sum(np.array(abs_corr)) - dia_sum
    # off_dia_sum = np.sum(abs_corr) - dia_sum

    # get number of off-diagonal elements
    off_dia_n = abs_corr.size - len(abs_corr)

    # calc average of off-diagonal elements
    off_dia_avg = off_dia_sum / off_dia_n

    return(off_dia_avg)


def get_diag_avg(df):

    # get array length
    df_len = len(df)
    
    # get indices of diagonal elements
    dia = np.diag_indices(df_len)

    # get sum of diagonal elements
    dia_sum = sum(np.array(df)[dia])
    
    # calculate avg
    dia_avg = dia_sum / df_len
    
    return dia_avg


def get_convergent_corrs(df):
    corrs = df[BFI_SCALE_IDS + IPIP_SCALE_IDS].corr().filter(items = IPIP_SCALE_IDS, axis = 0)[BFI_SCALE_IDS]
    return corrs


def get_model_level_convergent_corrs(df, model_id):
    model_df = df.query(f"model_id == '{model_id}'")
    corrs = get_convergent_corrs(model_df)
    return corrs


def get_avg_discriminant_corrs(df):
    convergent_corrs = get_convergent_corrs(df)
    avg_discriminant_corrs = get_avg_abs_discriminant_values(convergent_corrs)
    return avg_discriminant_corrs


def get_avg_convergent_corrs(df):
    convergent_corrs = get_convergent_corrs(df)
    avg_convergent_corrs = get_diag_avg(convergent_corrs)
    return avg_convergent_corrs


def get_remaining_row_col_vals(df, element_row_i, element_col_i):
    """Returns all other row values for a given element in a df"""
    df_len = len(df)
    vals = []
    
    for row_i in range(df_len):
        if row_i != element_row_i:
            vals.append(df.iloc[element_col_i, row_i])
    
    for col_i in range(df_len):
        if col_i != element_col_i:
            vals.append(df.iloc[col_i, element_row_i])
    
    return vals
            
    
def get_diagonal_indices(df):
    # get array length
    df_len = len(df)
    
    # get indices of diagonal elements
    dia = np.diag_indices(df_len)
    
    return dia    


def get_diffs(ref_val, comparison_vals):
    diffs = [ref_val - abs(off_val) for off_val in comparison_vals]
    return diffs


def get_avg_diff(ref_val, comparison_vals):
    # get difference between a reference value and absolute versions of comparison values
    diffs = get_diffs(ref_val, comparison_vals)
    return np.mean(diffs)


def get_diag_offdiag_diffs(df):
    # get diag indices
    dia = get_diagonal_indices(df)
    dia = list(map(list, zip(dia[0], dia[1])))
    
    # print(dia)
    
    all_diag_offdiag_diffs = []
        
    for i in range(len(df)):
        ref_val = df.iloc[i, i]
        comparison_vals = get_remaining_row_col_vals(df, i, i)
        diffs = get_diffs(ref_val, comparison_vals)
        
        all_diag_offdiag_diffs += diffs
        
    return all_diag_offdiag_diffs


def get_avg_diag_offdiag_diffs(df):
    diag_offdiag_diffs = get_diag_offdiag_diffs(df)
    return np.mean(diag_offdiag_diffs)

In [ ]:
pd.set_option('display.float_format', '{:.3f}'.format)
# subset discriminant correlations
mtmm = scored_joined[BFI_SCALE_IDS + IPIP_SCALE_IDS].corr()[IPIP_SCALE_IDS][0:5]
mtmm

In [ ]:
# subset convergent correlations
pd.DataFrame(np.diag(scored_joined[BFI_SCALE_IDS + IPIP_SCALE_IDS].corr().iloc[0:5,5:10])).transpose()

In [ ]:
# avg convergent correlation
get_avg_convergent_corrs(scored_joined)

In [ ]:
# avg discriminant correlation
get_avg_discriminant_corrs(scored_joined)

In [ ]:
convergent_corrs = get_model_level_convergent_corrs(
    scored_joined, MODEL_POINTER)
get_avg_diag_offdiag_diffs(convergent_corrs)

## IPIP-NEO-300 and Validation Scales

**Extraversion**
* Should correlate positively with PANAS Positive Affect.
* Should correlate negatively with NEU, PANAS Negative Affect.
* Discriminant validity: strongest positive correlate of PANAS Positive Affect

**Agreeableness**
* Should correlate negatively with BPAQ Physical Aggression, Verbal Aggression, Anger, and Hostility.
* Should correlate positively with PVQ-RR Conformity.
* Discriminant validity: 

**Conscientiousness**
* Should correlate positively with PVQ-RR broad values of Achievement, Conformity, Security.
* Discriminant validity: competes a bit with AGR in terms of relating to CONF, SCRT, but this might be because of suppressor effects. If we look at the CON facets of Orderliness (C2; likes order and regularlity, to tidy up), and Dutifulness (C3; following rules, keeping promises), and perhaps Cautiousness (C6; being careful, not doing crazy things), we will see stronger convergent correlations and better discriminant validity.

**Neuroticism**
* Should negatively correlate with EXT, AGR, CON, PANAS Positive Affect.
* Should positively correlate with PANAS Negative Affect.
* Should positively correlate with measures of aggression.

**Openness**
* Should positively correlate with SSCS Creative Self-Efficacy and SSCS Creative Personal Identity.
* (should be no or a negative correlation with ACHV, CONF, SCRT)

In [ ]:
scored_joined[IPIP_SCALE_IDS + VALIDATION_SCALE_IDS].corr().round(2)

In [ ]:
calculate_pvalues(scored_joined[IPIP_SCALE_IDS + VALIDATION_SCALE_IDS])

In [ ]:
# EXT
scored_joined \
    .groupby("model_id", sort=False) \
    [['IPIP300-EXT'] + ["PA", "NA"]] \
    .corr().round(2).unstack()['IPIP300-EXT'].T \
    .iloc[1:].rename_axis("Scale", axis=1)

In [ ]:
# AGR
scored_joined \
    .groupby("model_id", sort=False) \
    [['IPIP300-AGR'] + ["PHYS", "VRBL", "ANGR", "HSTL"]] \
    .corr().round(2).unstack()['IPIP300-AGR'].T \
    .iloc[1:].rename_axis("Scale", axis=1)

In [ ]:
# CON
scored_joined \
    .groupby("model_id", sort=False) \
    [['IPIP300-CON'] + ["ACHV", "CONF", "SCRT"]] \
    .corr().round(2).unstack()['IPIP300-CON'].T \
    .iloc[1:].rename_axis("Scale", axis=1)

In [ ]:
# NEU
scored_joined \
    .groupby("model_id", sort=False) \
    [['IPIP300-NEU'] + ["PA", "NA"]] \
    .corr().round(2).unstack()['IPIP300-NEU'].T \
    .iloc[1:].rename_axis("Scale", axis=1)

In [ ]:
# OPE
scored_joined \
    .groupby("model_id", sort=False) \
    [['IPIP300-OPE'] + ["CSE", "CPI"]] \
    .corr().round(2).unstack()['IPIP300-OPE'].T \
    .iloc[1:].rename_axis("Scale", axis=1)

## BFI and Validation Scales

In [ ]:
scored_joined[BFI_SCALE_IDS + VALIDATION_SCALE_IDS].corr().round(2)

In [ ]:
calculate_pvalues(scored_joined[BFI_SCALE_IDS + VALIDATION_SCALE_IDS])

# R Analysis

## Reliability Functionalized

In [ ]:
def launch_r_instance(psychometric_utils_path: str) -> None:
    # load R instance
    global r
    r = robjects.r

    # source R script
    r['source'](psychometric_utils_path)

    # load function(s) within script
    global tidyjson_r
    tidyjson_r = importr('tidyjson')
    # admin_session_to_nested_key_r = robjects.globalenv['admin_session_to_nested_key']
    # score_subscale_r = robjects.globalenv['score_subscale']
    
    global subscale_reliability_r
    subscale_reliability_r = robjects.globalenv['subscale_reliability']


def load_r_scored_session(scored_session_df: pd.DataFrame) -> pd.DataFrame:
    """Load scored_session_df in R."""
    with localconverter(robjects.default_converter + pandas2ri.converter):
      scored_session_df_r = robjects.conversion.py2rpy(scored_session_df)
    
    return scored_session_df_r

def compute_reliability_indices_per_scale(admin_session, admin_session_r, scored_session_df_r, **kwargs):   
    # create list of scores to be later converted into the output dataframe    
    score_list = []

    # compute reliability for each scale in an admin_session
    # if a particular reliability index can't be estimated, record as NA
    for measure_id, measure in admin_session.measures.items():
        for scale_id in measure.scales:

            # try computing Cronbach's Alpha
            try:
                alpha = subscale_reliability_r(admin_session_r, scored_session_df_r, measure_id, scale_id, "alpha")[0]
            except Exception as e:
                print(f"An error occurred while calculating alpha for measure {measure_id} and scale {scale_id}: {e}")
                alpha = np.nan

            # try computing McDonald's Omega
            try:
                omega = subscale_reliability_r(admin_session_r, scored_session_df_r, measure_id, scale_id, "omega")[0]
            except Exception as e:
                print(f"An error occurred while calculating omega for measure {measure_id} and scale {scale_id}: {e}")
                omega = np.nan

            # try computing Guttman's Lambda 6
            try:
                g6 = subscale_reliability_r(admin_session_r, scored_session_df_r, measure_id, scale_id, "G6")[0]
            except Exception as e:
                print(f"An error occurred while calculating G6 for measure {measure_id} and scale {scale_id}: {e}")
                g6 = np.nan

            # add the above reliability estimates to running score_list
            score_list.append([measure_id, scale_id, alpha, omega, g6])

    # combine accumulated estimates into one dataframe
    reliabilities_df = pd.DataFrame(score_list, columns=['measure_id', 'scale_id', 'alpha', 'omega', 'g6'])
    
    return reliabilities_df

def run_reliability_analysis_in_r(psychometric_utils_path: str,
                                  scored_session_df: pd.DataFrame,
                                  admin_session_json_path: str) -> pd.DataFrame:
    # launch R instance
    launch_r_instance(psychometric_utils_path)
    
    # load admin_session in R
    admin_session_r = tidyjson_r.read_json(admin_session_json_path)
    
    # load scored_session_df into R
    scored_session_df_r = load_r_scored_session(scored_session_df)
    
    # load main admin_session
    admin_session = survey_bench_lib.load_admin_session(
        admin_session_json_path)
    
    # compute reliability indices per scale
    reliabilities_df = compute_reliability_indices_per_scale(
        admin_session, admin_session_r, scored_session_df_r)
    
    return reliabilities_df

In [ ]:
reliabilities = run_reliability_analysis_in_r(
    psychometric_utils_path=PSYCHOMETRIC_UTILS_PATH,
    scored_session_df=scored_session_df,
    admin_session_json_path=ADMIN_SESSION_PATH
)

In [ ]:
reliabilities